In [526]:
import altair as alt
import pandas as pd
import sqlite3

In [527]:
id_disciplina = 1
id_professor = 3

In [528]:
print("Oi")

Oi


In [529]:
conn = sqlite3.connect('../silvia/db.sqlite3')
cur = conn.cursor()

In [530]:
query = f"""
SELECT
    a.id AS id_avaliacao, 
    s.carater , 
    a.intensidade, 
    d.id AS id_disciplina,
    p.id AS id_professor,
    p.nome_professor,
    assunto.id AS id_assunto,
    assunto.nome_assunto,
    t.id AS id_turma,
    t.curso,
    t.periodo,
    t.tag,
    a.data,
    a.conhecimento

FROM base_avaliacao AS a
INNER JOIN base_sentimentos AS s
ON a.id_sentimento_id = s.id

INNER JOIN base_disciplina as d
ON a.id_disciplina_id = d.id

INNER JOIN base_professor as p
ON d.id_professor_id = p.id

INNER JOIN base_assunto AS assunto
ON d.id_assunto_id = assunto.id

INNER JOIN base_turma AS t
ON d.id_turma_id = t.id

WHERE d.id={id_disciplina} 
        AND p.id={id_professor}
;
"""

grafico_barras = pd.read_sql_query(query, conn)

## 1 Grafico de barras

In [531]:
query = f"""
SELECT
    a.id AS id_avaliacao, 
    s.carater , 
    a.intensidade, 
    d.id AS id_disciplina,
    p.id AS id_professor,
    p.nome_professor,
    assunto.id AS id_assunto,
    assunto.nome_assunto,
    t.id AS id_turma,
    t.curso,
    t.periodo,
    t.tag,
    a.data,
    a.conhecimento

FROM base_avaliacao AS a
INNER JOIN base_sentimentos AS s
ON a.id_sentimento_id = s.id

INNER JOIN base_disciplina as d
ON a.id_disciplina_id = d.id

INNER JOIN base_professor as p
ON d.id_professor_id = p.id

INNER JOIN base_assunto AS assunto
ON d.id_assunto_id = assunto.id

INNER JOIN base_turma AS t
ON d.id_turma_id = t.id

WHERE d.id={id_disciplina} 
        AND p.id={id_professor}
;
"""

grafico_barras = pd.read_sql_query(query, conn)

In [532]:
colors = {"disposto-bom": '#A9CF54', 'disposto-ruim': '#F1433F', 'indisposto-bom': '#F7E967', 'indisposto-ruim': '#70B7BA'}
grafico_barras['color'] = grafico_barras['carater'].map(colors)

bar_graph = alt.Chart(grafico_barras.groupby(['color', 'carater']).sum().reset_index()).mark_bar().encode(
    alt.X('carater:N',sort='y'),
    alt.Y('intensidade:Q'),
    color=alt.Color('color:N', scale=None)
).properties(
    width=alt.Step(80)
)

bar_graph.save('grafico_barras.html', embed_options={'renderer':'svg'})

## 2 grafico de linha

In [533]:
id_disciplina = 1
id_professor = 3

In [534]:
query = f"""
SELECT
    a.id AS id_avaliacao, 
    s.carater , 
    a.intensidade, 
    d.id AS id_disciplina,
    p.id AS id_professor,
    p.nome_professor,
    assunto.id AS id_assunto,
    assunto.nome_assunto,
    t.id AS id_turma,
    t.curso,
    t.periodo,
    t.tag,
    a.data,
    a.conhecimento

FROM base_avaliacao AS a
INNER JOIN base_sentimentos AS s
ON a.id_sentimento_id = s.id

INNER JOIN base_disciplina as d
ON a.id_disciplina_id = d.id

INNER JOIN base_professor as p
ON d.id_professor_id = p.id

INNER JOIN base_assunto AS assunto
ON d.id_assunto_id = assunto.id

INNER JOIN base_turma AS t
ON d.id_turma_id = t.id

WHERE d.id={id_disciplina} 
        AND p.id={id_professor}
;
"""
grafico_linhas = pd.read_sql_query(query, conn)

In [535]:
for i in range(len(grafico_linhas)):
    if ('ruim' in grafico_linhas['carater'].loc[i]):
        grafico_linhas['intensidade'].loc[i] = (grafico_linhas['intensidade'].loc[i])*-1

c:\Users\raull\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [536]:
grafico_linhas.groupby(['data']).sum().reset_index().head(2)

,data,id_avaliacao,intensidade,id_disciplina,id_professor,id_assunto,id_turma,periodo,conhecimento
0,11/05/2022,3,-4,2,6,2,2,16,3
1,12/05/2022,7,2,2,6,2,2,16,7


In [537]:
# convert data into a datetime column
grafico_linhas['data'] = pd.to_datetime(grafico_linhas['data'], format='%d/%m/%Y')
grafico_linhas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_avaliacao    24 non-null     int64         
 1   carater         24 non-null     object        
 2   intensidade     24 non-null     int64         
 3   id_disciplina   24 non-null     int64         
 4   id_professor    24 non-null     int64         
 5   nome_professor  24 non-null     object        
 6   id_assunto      24 non-null     int64         
 7   nome_assunto    24 non-null     object        
 8   id_turma        24 non-null     int64         
 9   curso           24 non-null     object        
 10  periodo         24 non-null     int64         
 11  tag             24 non-null     object        
 12  data            24 non-null     datetime64[ns]
 13  conhecimento    24 non-null     int64         
dtypes: datetime64[ns](1), int64(8), object(5)
memory usage: 2.8+

In [538]:
alt.Chart(grafico_linhas.groupby(['data']).mean().reset_index()).mark_bar().encode(
    alt.X('data:T', title='Data', axis=alt.Axis(format='%d/%m/%Y')),
    alt.Y('intensidade:Q'),
    color=alt.condition(
        alt.datum.intensidade > 0,
        alt.value('#A9CF54'),
        alt.value('#F1433F')
    )
)

alt.Chart(...)

____________

## 3) histograma

In [539]:
query = f"""
SELECT
    a.id AS id_avaliacao, 
    s.carater , 
    a.intensidade, 
    d.id AS id_disciplina,
    p.id AS id_professor,
    p.nome_professor,
    assunto.id AS id_assunto,
    assunto.nome_assunto,
    t.id AS id_turma,
    t.curso,
    t.periodo,
    t.tag,
    a.data,
    a.conhecimento

FROM base_avaliacao AS a
INNER JOIN base_sentimentos AS s
ON a.id_sentimento_id = s.id

INNER JOIN base_disciplina as d
ON a.id_disciplina_id = d.id

INNER JOIN base_professor as p
ON d.id_professor_id = p.id

INNER JOIN base_assunto AS assunto
ON d.id_assunto_id = assunto.id

INNER JOIN base_turma AS t
ON d.id_turma_id = t.id

WHERE d.id={id_disciplina} 
        AND p.id={id_professor}
;
"""

histograma = pd.read_sql_query(query, conn)
histograma.head(2)

,id_avaliacao,carater,intensidade,id_disciplina,id_professor,nome_professor,id_assunto,nome_assunto,id_turma,curso,periodo,tag,data,conhecimento
0,1,disposto-bom,1,1,3,Marcos,1,portugues,1,fundamental_1,8,B,11/05/2022,1
1,2,indisposto-ruim,5,1,3,Marcos,1,portugues,1,fundamental_1,8,B,11/05/2022,2


In [540]:
histogram = alt.Chart(histograma,width=500, height=350).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'
).mark_bar().encode(
    alt.X('conhecimento:Q', bin=True, title='Conhecimento categorizado'),
    alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title='porcentagem'),
).configure_bar(
    color = '#70B7BA'
)

# change the title of x and y axis
histogram.save('histograma.html', embed_options={'renderer': 'svg'})

In [ ]:
conn.commit()
conn.close()